# CS110 Assignment 1: K-Way Merge Sort
-------------------------

To perform analysis on different sorting algorithms, I first made a function to generate unsorted arrays to sort. It takes as input a length, and also a value that determines the type of array: either randomly shuffled, to analyze the performance in the average case; ordered, to analyze performance in the best case; or reversed, to analyze performance in the worst case:

In [1]:
from random import shuffle

#generate unsorted array - Theta(n)
def gen_array(length, case=0): #0 for random, 1 for best case, 2 for worst case
    array = range(0,length) #O(n)
    if case==0:
        shuffle(array) #O(n)
    if case==2:
        array.reverse() #O(n)
    return array

I decided to go tackle the optional challenge, and devise a k-way merge-sort algorithm. For that end, I needed a k-way merge function, that would take k ordered arrays, and combine them into one. The algorithm is explained in the comments below:

In [2]:
from math import ceil

#k-way merge, takes list of lists to merge to one - Theta(k^2n^2)+Theta(k)
def merge(arrays):
    #create empty list to sort into
    output = []
    #main loop runs until the list of lists is empty
    while arrays: #O(kn^2) + O(nk^2)+O(k) since it runs once for every element (kn), and takes O(k)+O(n) for each, plus removing each array once for O(k). 
        #initialization to first remaining element of first remaining list
        i = 0
        minimum = arrays[0][0]
        #check first element of each list in the list of lists, as they are all assumed to be sorted
        for array in arrays: #O(k)
            #find the minimum value and get its list index
            if array[0]<minimum:
                i = arrays.index(array) #O(1)
                minimum = array[0]
        #remove minimum value from the original list and add it to the sorted list
        arrays[i].pop(0) #O(n)! This is a problem.
        output.append(minimum) #O(1)
        #if we emptied a list, remove it from the list of lists
        if not arrays[i]:
            arrays.remove(arrays[i]) #O(k)
    #return sorted list
    return output        

When researching the complexity of the pre-built functions I was using, I noticed that I was calling the list.pop() function to remove the *first* element from an array.

The complexity to pop the last element of an array is O(1), but popping any other element requires a search down the array, and a reordering of the rest of the array, with complexity O(n). Since this happens whenever I enter a value into the sorted array, it happens once for every element in the sorted array, and so the complexity multiplies to O(n^2).

I decided to try and mitigate this, by using markers to keep track of the elements that have been sorted, instead of removing elements from the arrays.

This results in a another operation that depends on the amount of arrays merged (which I dubbed O(k)), but we can assume that while the number of elements to sort, n, can get huge, we will keep k reasonably low, I predicted that O(k) will not make much of a difference. I decided to keep my original merge function, to see how my prediction fares in the analysis to follow:

In [3]:
#improved k-way merge, without .pop(0) - Theta(n)
#basically, instead of removing elements from the arrays until they're empty,
#we keep track of the elements we've already sorted using markers
def merge2(arrays): #O(k^2n)+O(2k)
    output = []
    #we initiallize markers with a 0 at the end of each array
    for array in arrays: #O(k)
        array.append(0) #O(1)
    while arrays: #O(nk^2)+O(k) since it runs once for every element (kn), and takes O(k) for each, plus removing each array once for O(k). 
        #we initialize not to the first element of each array, but to the element designated by the marker for that array
        i = 0
        minimum = arrays[0][arrays[0][-1]]
        for array in arrays:
            if array[array[-1]]<minimum:
                i = arrays.index(array)
                minimum = array[array[-1]]
        #we increment our marker instead of removing an element
        arrays[i][-1] = arrays[i][-1]+1 #O(1), hurray!
        output.append(minimum)
        if len(arrays[i])<=arrays[i][-1]+1:
            arrays.remove(arrays[i]) #O(k)
    return output

In order to complete the assignment requirements, I needed to implement insertion-sort, which we have already done for class. Finally, I moved on to creating the merge-sort function that separates an array recursively and calls on merge to return a sorted list.

Apart from the array to be sorted, my function takes a variable k to determine the k-way sort structure, a variable "modified" that is set to 0 if we want to use pure merge-sort or any number to set the minimum array length for insertion-sort to override the precedure, and a boolean variable "improved" to choose between the two versions of the merge function:

In [4]:
#insertion-sort, from pre-class work - O(n^2), Omega(n)
def insertion_sort(array):
    for i in range(1, len(array)):
        key = array[i]
        j = i-1
        while j >=0 and key < array[j] :
                array[j+1] = array[j]
                j -= 1
        array[j+1] = key
    return array


#k-way merge-sort - Complexity depends on parameters as well as input. Will be expanded upon later.
def merge_sort(array, k=2, modified=0, improved=False): #input minimum length for merge-sort for the modified algorithm. <k+1 doesn't make sense but works.
    #check for trivial exception
    if len(array) == 1:
        return array
    #if we selected the modified version, we defer to insertion-sort instead
    if modified!=0 and len(array)>modified:
        return insertion_sort(array)
    
    #We split the list into k sublists by skipping over j=length/k elements when we take the range() function below.
    #We need a float for the division and an int for range(), so we juggle data types a bit:
    j = int(ceil(float(len(array))/k)) #O(1)

    #Here, we get a list of lists to merge, by recursively splitting the list into k sublists each time, skipping over j elements.
    arrays = [merge_sort(array[i:i+j], k, modified, improved) for i in range(0, len(array), j)]
    #This command merges k lists to prepare them for the next level of merging, or to finish the algorithm.
    if improved==False:
        return merge(arrays)
    return merge2(arrays)

I've listed the complexity of my various sub-function along the way, but now it's time to analyze the complexity of the different versions of my merge-sort function.

The function lets us choose between:
1. A merge function with O(k^2n^2), or an improved merge function with O(k^2n). This value will be multiplied by the amount of recursive steps we take.

2. Any value of k from 2 to n, into which to separate the array at each stage. We have proved in class that this value will impact the amount of recursive steps taken, so that it equals log_k(n).

3. A pure merge-sort vs a modified merge-sort with insertion sort kicking in at any value from 1 to n, which effectively means any value from k+1 (since below that limit, the arrays encountered will be of length 1, which is trivially O(1) in insertion-sort) and n/2 (since any value larger will only get recognized after the first separation of the array, and since a value of n would be a trivial insertion-sort). This will result in a reduced amount of recursive steps, with the last step replaced by insertion-sort, which is an O(n^2) function. Since k represents the 

When we merge our function using pure merge-sort with our improved algorithm, for each of the log_k(n) levels (denoted by j) in our recursive tree, we have k^j groups of n/k^j elements. So, for every level the amount of computations we have to carry out is:

k^j(k^2n/k^j) = k^2n

And so, we have a constant number of operations for each stage, and we can solve for the complexity of the recursion by multiplying by the number of stages. Therefore, for pure merge-sort with the improved algorithm: O(nk^2log_k(n))

For the modified merge-sort, we cut steps and introduce an insertion-sort step of complexity O(n^2). Since the length of an array in level j is n/k^j, then the amount of levels skipped by a setting a minimum merge-sort length at m is log_k(n/m)+1.
As we've proved above, an operation (in this case of complexity O(n^2)) will retain it's compexity when applied to k^j groups of n/k^j elements.
Therefore, we can use the complexity of pure merge sort with that many less levels, and add an instance of O(n^2). So, the complexity of the modified, improved, k-way merge-sort is: O(nk^2(log_k(n)-log_k(n/m)))+O(n^2) = O(nk^2log_k(m))+O(n^2).


Now we can finally move on to empirical analysis. I'll run a few loops, five times each, to test the average running time of the algorithm under different situations:
1. With n={10,1000,10000}
2. With k={2,5,10}
3. With m={0,3,10}
4. With my old and new merge functions.
5. With random, best and worst case data.

I saved the results into a csv file to present them nicely.

In [ ]:
import time
import csv

with open('results.csv', 'wb') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['n','k','m','d','i','t'])
    for n in [10,1000,10000]:
        for k in [2,5,10]:
            for m in [0,3,10]:
                for d in [0,1,2]:
                    total = 0
                    for i in range(5):
                        array = gen_array(n,d)
                        start = time.time()
                        merge_sort(array,k,m,True)
                        total = total + time.time()-start
                    filewriter.writerow([n,k,m,d,'True',total/100])
                    total = 0
                    for i in range(5):
                        array = gen_array(n,d)
                        start = time.time()
                        merge_sort(array,k,m,False)
                        total = total + time.time()-start
                    filewriter.writerow([n,k,m,d,'False',total/100])

Upon inspection it was clear that for in all cases, sorting ten elements was too quick for my time keeping method, which returned mostly zeros, and nothing slower than a hundred thousandth of a second, and so the data was discarded.

For both n=1000 and n=10000, our improved algorithm performed much better on average. This was expected, since k>>n.
However, it seemed that running time was still dominated by the effect of the scenario. Best case scenario averages were ten times to a hundrend times as quick as random scenarios. To examine this, I decided to look at statistics for each scenario.

It has become apparent that for a random and worst scenarios, our chosen values of k did not make much of a difference on performance times.
However, we reached siginificant improvement in the best case scenario by using a higher k value. This is due to the fact that with a higher k, less splits were made, and less merges were called - these invlove extra computation on top of the trivial merge precedure.

The modified version using insertion sort was very bad news for our run-time on random and worst case scenarios, increasing the duration by two orders of magnitude. However, it decreased the duration similarly for best case scenarios, since insertion sort is faster than merge of an already sorted array. In both cases, the exact value of our limit did not make much of a difference, but it's easily predicted that for the best case scenario, an arbitrarily high limit of m=n would be quickest since it means using insertion sort on the entire array - an algorithm with Omega(n).

In conclusion, it seems that there is no best variation for our algorithm, since the effect of the key variables depends on the nature of the input. For partially sorted arrays, we would be advised to use a modified, high-k version. However, when such an expectation is not feasible, we should avoid insertion sort.

![title](kway.PNG)